In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
%matplotlib inline

In [2]:
# read training data 
train_df = pd.read_csv('PM_train_01.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [3]:
# read test data
test_df = pd.read_csv('PM_test_01.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [4]:
# read ground truth data
truth_df = pd.read_csv('PM_truth_01.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [5]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1   -0.0007   -0.0004     100.0  518.67  641.82  1589.70  1400.60   
1   1      2    0.0019   -0.0003     100.0  518.67  642.15  1591.82  1403.14   
2   1      3   -0.0043    0.0003     100.0  518.67  642.35  1587.99  1404.20   
3   1      4    0.0007    0.0000     100.0  518.67  642.35  1582.79  1401.87   
4   1      5   -0.0019   -0.0002     100.0  518.67  642.37  1582.85  1406.22   

      s5   ...        s12      s13      s14     s15   s16  s17   s18    s19  \
0  14.62   ...     521.66  2388.02  8138.62  8.4195  0.03  392  2388  100.0   
1  14.62   ...     522.28  2388.07  8131.49  8.4318  0.03  392  2388  100.0   
2  14.62   ...     522.42  2388.03  8133.23  8.4178  0.03  390  2388  100.0   
3  14.62   ...     522.86  2388.08  8133.83  8.3682  0.03  392  2388  100.0   
4  14.62   ...     522.19  2388.04  8133.80  8.4294  0.03  393  2388  100.0   

     s20      s21  
0  39.06  23.4190  
1  39.00  23.4236  
2  38.95  23.3442  
3  38.88  23.3739  
4  38.90  23.4044  

[5 rows x 26 columns]

In [6]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1   -0.0007   -0.0004     100.0  518.67  641.82  1589.70  1400.60   
1   1      2    0.0019   -0.0003     100.0  518.67  642.15  1591.82  1403.14   
2   1      3   -0.0043    0.0003     100.0  518.67  642.35  1587.99  1404.20   
3   1      4    0.0007    0.0000     100.0  518.67  642.35  1582.79  1401.87   
4   1      5   -0.0019   -0.0002     100.0  518.67  642.37  1582.85  1406.22   

      s5 ...       s13      s14     s15   s16  s17   s18    s19    s20  \
0  14.62 ...   2388.02  8138.62  8.4195  0.03  392  2388  100.0  39.06   
1  14.62 ...   2388.07  8131.49  8.4318  0.03  392  2388  100.0  39.00   
2  14.62 ...   2388.03  8133.23  8.4178  0.03  390  2388  100.0  38.95   
3  14.62 ...   2388.08  8133.83  8.3682  0.03  392  2388  100.0  38.88   
4  14.62 ...   2388.04  8133.80  8.4294  0.03  393  2388  100.0  38.90   

       s21  RUL  
0  23.4190  191  
1  23.4236  190  
2  23.3442  189  
3  23.3739  188  
4  23.4044  187  

[5 rows x 27 columns]

In [7]:
# generate label columns for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1   -0.0007   -0.0004     100.0  518.67  641.82  1589.70  1400.60   
1   1      2    0.0019   -0.0003     100.0  518.67  642.15  1591.82  1403.14   
2   1      3   -0.0043    0.0003     100.0  518.67  642.35  1587.99  1404.20   
3   1      4    0.0007    0.0000     100.0  518.67  642.35  1582.79  1401.87   
4   1      5   -0.0019   -0.0002     100.0  518.67  642.37  1582.85  1406.22   

      s5   ...       s15   s16  s17   s18    s19    s20      s21  RUL  label1  \
0  14.62   ...    8.4195  0.03  392  2388  100.0  39.06  23.4190  191       0   
1  14.62   ...    8.4318  0.03  392  2388  100.0  39.00  23.4236  190       0   
2  14.62   ...    8.4178  0.03  390  2388  100.0  38.95  23.3442  189       0   
3  14.62   ...    8.3682  0.03  392  2388  100.0  38.88  23.3739  188       0   
4  14.62   ...    8.4294  0.03  393  2388  100.0  38.90  23.4044  187       0   

   label2  
0       0  
1       0  
2       0  
3       0  
4       0  

[5 rows x 29 columns]

In [8]:
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.459770  0.166667       0.0  0.0  0.183735  0.406802  0.309757   
1   1      2  0.609195  0.250000       0.0  0.0  0.283133  0.453019  0.352633   
2   1      3  0.252874  0.750000       0.0  0.0  0.343373  0.369523  0.370527   
3   1      4  0.540230  0.500000       0.0  0.0  0.343373  0.256159  0.331195   
4   1      5  0.390805  0.333333       0.0  0.0  0.349398  0.257467  0.404625   

    s5     ...      s16       s17  s18  s19       s20       s21  RUL  label1  \
0  0.0     ...      0.0  0.333333  0.0  0.0  0.713178  0.724662  191       0   
1  0.0     ...      0.0  0.333333  0.0  0.0  0.666667  0.731014  190       0   
2  0.0     ...      0.0  0.166667  0.0  0.0  0.627907  0.621375  189       0   
3  0.0     ...      0.0  0.333333  0.0  0.0  0.573643  0.662386  188       0   
4  0.0     ...      0.0  0.416667  0.0  0.0  0.589147  0.704502  187       0   

   label2  cycle_norm  
0       0     0.00000  
1       0     0.00277  
2       0     0.00554  
3       0     0.00831  
4       0     0.01108  

[5 rows x 30 columns]

In [9]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.632184  0.750000       0.0  0.0  0.545181  0.310661  0.269413   
1   1      2  0.344828  0.250000       0.0  0.0  0.150602  0.379551  0.222316   
2   1      3  0.517241  0.583333       0.0  0.0  0.376506  0.346632  0.322248   
3   1      4  0.741379  0.500000       0.0  0.0  0.370482  0.285154  0.408001   
4   1      5  0.580460  0.500000       0.0  0.0  0.391566  0.352082  0.332039   

    s5     ...           s13       s14       s15  s16       s17  s18  s19  \
0  0.0     ...      0.220588  0.132160  0.308965  0.0  0.333333  0.0  0.0   
1  0.0     ...      0.264706  0.204768  0.213159  0.0  0.416667  0.0  0.0   
2  0.0     ...      0.220588  0.155640  0.458638  0.0  0.416667  0.0  0.0   
3  0.0     ...      0.250000  0.170090  0.257022  0.0  0.250000  0.0  0.0   
4  0.0     ...      0.220588  0.152751  0.300885  0.0  0.166667  0.0  0.0   

        s20       s21  cycle_norm  
0  0.558140  0.661834     0.00000  
1  0.682171  0.686827     0.00277  
2  0.728682  0.721348     0.00554  
3  0.666667  0.662110     0.00831  
4  0.658915  0.716377     0.01108  

[5 rows x 27 columns]

In [10]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [11]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.632184  0.750000       0.0  0.0  0.545181  0.310661  0.269413   
1   1      2  0.344828  0.250000       0.0  0.0  0.150602  0.379551  0.222316   
2   1      3  0.517241  0.583333       0.0  0.0  0.376506  0.346632  0.322248   
3   1      4  0.741379  0.500000       0.0  0.0  0.370482  0.285154  0.408001   
4   1      5  0.580460  0.500000       0.0  0.0  0.391566  0.352082  0.332039   

    s5 ...        s14       s15  s16       s17  s18  s19       s20       s21  \
0  0.0 ...   0.132160  0.308965  0.0  0.333333  0.0  0.0  0.558140  0.661834   
1  0.0 ...   0.204768  0.213159  0.0  0.416667  0.0  0.0  0.682171  0.686827   
2  0.0 ...   0.155640  0.458638  0.0  0.416667  0.0  0.0  0.728682  0.721348   
3  0.0 ...   0.170090  0.257022  0.0  0.250000  0.0  0.0  0.666667  0.662110   
4  0.0 ...   0.152751  0.300885  0.0  0.166667  0.0  0.0  0.658915  0.716377   

   cycle_norm  RUL  
0     0.00000  142  
1     0.00277  141  
2     0.00554  140  
3     0.00831  139  
4     0.01108  138  

[5 rows x 28 columns]

In [12]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.632184  0.750000       0.0  0.0  0.545181  0.310661  0.269413   
1   1      2  0.344828  0.250000       0.0  0.0  0.150602  0.379551  0.222316   
2   1      3  0.517241  0.583333       0.0  0.0  0.376506  0.346632  0.322248   
3   1      4  0.741379  0.500000       0.0  0.0  0.370482  0.285154  0.408001   
4   1      5  0.580460  0.500000       0.0  0.0  0.391566  0.352082  0.332039   

    s5   ...    s16       s17  s18  s19       s20       s21  cycle_norm  RUL  \
0  0.0   ...    0.0  0.333333  0.0  0.0  0.558140  0.661834     0.00000  142   
1  0.0   ...    0.0  0.416667  0.0  0.0  0.682171  0.686827     0.00277  141   
2  0.0   ...    0.0  0.416667  0.0  0.0  0.728682  0.721348     0.00554  140   
3  0.0   ...    0.0  0.250000  0.0  0.0  0.666667  0.662110     0.00831  139   
4  0.0   ...    0.0  0.166667  0.0  0.0  0.658915  0.716377     0.01108  138   

   label1  label2  
0       0       0  
1       0       0  
2       0       0  
3       0       0  
4       0       0  

[5 rows x 30 columns]

In [13]:
def gen_label(df):
    y = df['label1']
    df.drop(['label1','label2','RUL'],axis=1,inplace=True)
    return y,df

In [14]:
y_train,X_train = gen_label(train_df)
print(y_train.shape,X_train.shape)
y_test,X_test = gen_label(test_df)
print(y_test.shape,X_test.shape)

(20631,) (20631, 27)
(13096,) (13096, 27)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.075, random_state=42)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(C=40)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
print('acc for val: ',accuracy_score(y_val,y_pred))

acc for val:  0.9683462532299741


In [17]:
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_val, y_pred)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[1314,   19],
       [  30,  185]])

In [18]:
# compute precision and recall
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.9068627450980392 
 recall =  0.8604651162790697


In [19]:
y_pred_test = lr.predict(X_test)
print('acc for test: ',accuracy_score(y_test,y_pred_test))

acc for test:  0.9858735491753207


In [20]:
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12715,    49],
       [  136,   196]])

In [21]:
# compute precision and recall
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  0.8 
 Recall:  0.5903614457831325 
 F1-score: 0.6793760831889082


In [22]:
results_df = pd.DataFrame([[accuracy_score(y_test,y_pred_test),precision_test,recall_test,f1_test],],
                         columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'],
                         index = ['logistic'])
results_df

Accuracy  Precision    Recall  F1-score
logistic  0.985874        0.8  0.590361  0.679376